In [1]:
from google.colab import files
import csv
import zipfile
import random
from datetime import datetime

In [2]:
# Configuración
FECHA_ACTUAL = datetime.now().strftime('%d/%m/%Y')
ARCHIVOS = {
    'Establecimientos.csv': [
        'FECHA', 'CORREO', 'RAZON_SOCIAL', 'NUMERO_RUC', 'NOMBRE_ESTABLECIMIENTO',
        'ACTIVIDAD_PRINCIPAL', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO',
        'DNI_RESPONSABLE', 'NOMBRE_RESPONSABLE', 'TELEFONO_RESPONSABLE',
        'CONSUMO_ELECTRICO_WATT', 'CONSUMO_AGUA_PROCESO_L', 'CONSUMO_AGUA_OFICINAS_L',
        'AGUAS_RESIDUALES', 'UNIDAD_MEDIDA', 'METODO_CALCULO'
    ],
    'HC1_Ubicacion.csv': [
        'FECHA', 'CORREO', 'RAZON_SOCIAL', 'NUMERO_RUC', 'NOMBRE_ESTABLECIMIENTO',
        'HC1-Punto', 'HC1-Zona', 'HC1-Este', 'HC1-Norte', 'HC1-Region', 'HC1-Referencia'
    ],
    'HC2_Combustibles.csv': [
        'FECHA', 'CORREO', 'RAZON_SOCIAL', 'NUMERO_RUC', 'NOMBRE_ESTABLECIMIENTO',
        'HC2-Fuente', 'HC2-Tipo_combustible', 'HC2-Cantidad_toneladas', 'HC2-Densidad_KgL'
    ],
    'HC3_Chimeneas.csv': [
        'FECHA', 'CORREO', 'RAZON_SOCIAL', 'NUMERO_RUC', 'NOMBRE_ESTABLECIMIENTO',
        'HC3-Chimenea', 'HC3-Presion_atm', 'HC3-Temperatura_C', 'HC3-Porcentaje',
        'HC3-Flujo_real_m3min', 'HC3-Flujo_normal_m3min', 'HC3-Horas_ano'
    ],
    'HC4_Descargas.csv': [
        'FECHA', 'CORREO', 'RAZON_SOCIAL', 'NUMERO_RUC', 'NOMBRE_ESTABLECIMIENTO',
        'HC4-Ducto', 'HC4-Caudal_Ls', 'HC4-Horas_ano', 'HC4-Tipo', 'HC4-Fluido', 'HC4-Presion_bar'
    ],
    'HC5_Emisiones.csv': [
        'FECHA', 'CORREO', 'RAZON_SOCIAL', 'NUMERO_RUC', 'NOMBRE_ESTABLECIMIENTO',
        'HC5-Sustancia', 'HC5-Cuerpo_receptor', 'HC5-Nombre_cuerpo', 'HC5-Unidad', 'HC5-Cantidad', 'HC5-Metodo_calculo'
    ],
    'HC6_Residuos.csv': [
        'FECHA', 'CORREO', 'RAZON_SOCIAL', 'NUMERO_RUC', 'NOMBRE_ESTABLECIMIENTO',
        'Residuo', 'Cantidad_transferida', 'Unidad_medida', 'Densidad_KgL',
        'RUC_operador', 'Tipo_operador', 'Numero_registro'
    ]
}

In [3]:
# Función para subir el archivo a Colab
def upload_file():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'Archivo {filename} cargado ({len(uploaded[filename])} bytes)')
    return list(uploaded.keys())[0] if uploaded else None

In [4]:
# Leer datos originales del TXT
def leer_empresas(filename):
    empresas = []
    with open(filename, 'r', encoding='utf-8') as f:
        lineas = f.readlines()[1:]  # Saltar encabezado

        for linea in lineas:
            datos = linea.strip().split(',')
            try:
                empresas.append({
                    'Fecha': datos[0],
                    'Email': datos[1],
                    'RazonSocial': datos[2],
                    'RUC': datos[3],
                    'CIIU': datos[4],
                    'Establecimientos': int(datos[8])
                })
            except IndexError as e:
              print(f"Error en línea: {linea} - {str(e)}")
              continue
    return empresas

In [6]:
# Generar datos tecnicos segun actividad (CIIU)
def generar_datos_tecnicos(ciiu):
    # Sector Alimentos y Bebidas
    if '10' in ciiu[:2]:  # Elaboración de productos alimenticios
        return {
            'actividad': "Elaboracion de productos alimenticios",
            'residuos': ['Residuos organicos', 'Envases contaminados', 'Subproductos animales'],
            'combustibles': ['Gas natural', 'GLP', 'Biomasa'],
            'emisiones': ['CO2', 'VOCs', 'Particulas'],
            'descargas': ['Aguas residuales', 'Liquidos de proceso'],
            'fluidos': ['Agua', 'Aceite vegetal'],
            'unidades': ['toneladas', 'kg', 'litros'],
            'densidades': [0.5, 1.2, 0.9]
        }

    # Sector Textil
    elif '13' in ciiu[:2]:  # Fabricación de textiles
        return {
            'actividad': "Fabricacion de productos textiles",
            'residuos': ['Recortes textiles', 'Colorantes', 'Lodos quimicos'],
            'combustibles': ['Diesel', 'Electricidad', 'Gas natural'],
            'emisiones': ['Formaldehido', 'PM10', 'COV'],
            'descargas': ['Aguas con tintes', 'Aguas residuales'],
            'fluidos': ['Agua', 'Solventes'],
            'unidades': ['toneladas', 'kg', 'm3'],
            'densidades': [0.3, 1.1, 0.8]
        }

    # Sector Químico
    elif '20' in ciiu[:2]:  # Industria química
        subcat = {
            '201': "Fabricacion de productos quimicos basicos",
            '202': "Fabricacion de pesticidas",
            '203': "Fabricacion de pinturas"
        }.get(ciiu[:3], "Fabricacion de productos quimicos")

        return {
            'actividad': subcat,
            'residuos': ['Lodos quimicos', 'Envases peligrosos', 'Catalizadores usados'],
            'combustibles': ['Fuel oil', 'Gas LP', 'Coque'],
            'emisiones': ['VOCs', 'NH3', 'SO2', 'HCl'],
            'descargas': ['Efluentes quimicos', 'Aguas de proceso'],
            'fluidos': ['Acidos', 'Solventes', 'Agua'],
            'unidades': ['toneladas', 'kg', 'barriles'],
            'densidades': [1.2, 0.8, 1.5]
        }

    # Sector Metalúrgico
    elif '24' in ciiu[:2]:  # Fabricación de productos metálicos
        subcat = {
            '241': "Industria siderurgica",
            '242': "Fabricacion de productos metalicos",
            '243': "Fundicion de metales"
        }.get(ciiu[:3], "Fabricacion de productos metalicos")

        return {
            'actividad': subcat,
            'residuos': ['Virutas metalicas', 'Aceites usados', 'Escorias'],
            'combustibles': ['Carbon', 'Coque', 'Gas natural'],
            'emisiones': ['PM10', 'CO', 'NOx', 'Metales pesados'],
            'descargas': ['Aguas de enfriamiento', 'Aguas con aceites'],
            'fluidos': ['Aceite industrial', 'Agua', 'Metal fundido'],
            'unidades': ['toneladas', 'kg', 'm3'],
            'densidades': [2.5, 0.9, 1.1]
        }

    # Sector Minería
    elif '07' in ciiu[:2]:  # Extracción de minerales
        subcat = {
            '071': "Extraccion de minerales metaliferos",
            '072': "Extraccion de minerales no metaliferos",
            '081': "Extraccion de piedra, arena y arcilla"
        }.get(ciiu[:3], "Extraccion de minerales")

        return {
            'actividad': subcat,
            'residuos': ['Relaves mineros', 'Escombreras', 'Lodos'],
            'combustibles': ['Diesel', 'GLP', 'Electricidad'],
            'emisiones': ['PM2.5', 'SO2', 'Particulas'],
            'descargas': ['Aguas acidas', 'Aguas de mina'],
            'fluidos': ['Agua', 'Suspensiones minerales'],
            'unidades': ['toneladas', 'm3', 'kg'],
            'densidades': [1.8, 1.2, 0.7]
        }

    # Sector Construcción
    elif '23' in ciiu[:2]:  # Fabricación de productos minerales no metálicos
        return {
            'actividad': "Fabricacion de cemento, cal y yeso",
            'residuos': ['Polvos de horno', 'Escorias', 'Material refractario'],
            'combustibles': ['Coque de petroleo', 'Carbon', 'Neumaticos usados'],
            'emisiones': ['CO2', 'PM10', 'NOx', 'SO2'],
            'descargas': ['Aguas de refrigeracion', 'Aguas residuales'],
            'fluidos': ['Agua', 'Lodos'],
            'unidades': ['toneladas', 'kg', 'm3'],
            'densidades': [1.4, 0.9, 1.3]
        }

    # Sector Energía
    elif '35' in ciiu[:2]:  # Suministro de energía eléctrica
        return {
            'actividad': "Generacion de energia electrica",
            'residuos': ['Cenizas', 'Lodos', 'Aceites dieléctricos'],
            'combustibles': ['Gas natural', 'Diesel', 'Carbon'],
            'emisiones': ['CO2', 'NOx', 'SO2', 'Material particulado'],
            'descargas': ['Aguas de refrigeracion', 'Aguas de purga'],
            'fluidos': ['Agua', 'Vapor'],
            'unidades': ['toneladas', 'kg', 'litros'],
            'densidades': [1.1, 0.8, 0.9]
        }

    # Sector Papel
    elif '17' in ciiu[:2]:  # Fabricación de papel
        return {
            'actividad': "Fabricacion de papel y productos de papel",
            'residuos': ['Lodos de pulpa', 'Recortes', 'Productos quimicos'],
            'combustibles': ['Biomasa', 'Gas natural', 'Fuel oil'],
            'emisiones': ['CO2', 'TRS', 'Particulas'],
            'descargas': ['Aguas residuales', 'Licor negro'],
            'fluidos': ['Agua', 'Pulpa'],
            'unidades': ['toneladas', 'kg', 'm3'],
            'densidades': [0.6, 1.0, 1.2]
        }

    # Default para otros sectores
    else:
        return {
            'actividad': "Actividad industrial general",
            'residuos': ['Residuos solidos', 'Lodos', 'Materiales varios'],
            'combustibles': ['Diesel', 'Electricidad', 'GLP'],
            'emisiones': ['CO2', 'PM2.5', 'CO'],
            'descargas': ['Aguas residuales', 'Aguas servidas'],
            'fluidos': ['Agua', 'Aire comprimido'],
            'unidades': ['toneladas', 'kg', 'm3'],
            'densidades': [1.0, 0.8, 1.1]
        }



In [17]:
# Generar archivos CSV
def generar_csvs(empresas):
    datos = {archivo: [] for archivo in ARCHIVOS}

    for empresa in empresas[:200]:
        datos_tecnicos = generar_datos_tecnicos(empresa['CIIU'])

        for i in range(1, empresa['Establecimientos'] + 1):
            nombre_estab = f"Establecimiento_{i}_{empresa['RUC'][-4:]}"

            # Datos comunes para todos los registros
            datos_base = [
                empresa['Fecha'],
                empresa['Email'],
                empresa['RazonSocial'],
                empresa['RUC'],
                nombre_estab
            ]

            # Establecimientos.csv
            datos['Establecimientos.csv'].append(datos_base + [
                datos_tecnicos['actividad'],
                "Lima",  # Departamento
                "Lima" if random.random() > 0.3 else "Callao",  # Provincia
                random.choice(["San Isidro", "Surco", "La Molina", "Lince"]),  # Distrito
                f"{random.randint(40000000, 49999999)}",  # DNI
                f"Responsable_{nombre_estab.replace(' ', '_')}",
                f"9{random.randint(1000000, 9999999)}",  # Teléfono
                random.randint(100000, 5000000),  # Consumo eléctrico
                random.randint(50000, 2000000),  # Agua proceso
                random.randint(10000, 200000),   # Agua oficinas
                round(random.uniform(100, 5000), 2),  # Aguas residuales
                "m3",
                random.choice(["Medicion directa", "Estimacion", "Balance hidrico"])
            ])

            # HC1_Ubicacion.csv
            datos['HC1_Ubicacion.csv'].append(datos_base + [
                f"P-{i:03d}",
                "Zona_Industrial",
                random.randint(280000, 290000),
                random.randint(8650000, 8660000),
                "Lima",
                f"Km_{random.randint(1, 50)}_Carretera_Central"
            ])

            # HC2_Combustibles.csv (2 registros)
            for _ in range(2):
                combustible = random.choice(datos_tecnicos['combustibles'])
                datos['HC2_Combustibles.csv'].append(datos_base + [
                    random.choice(["Horno", "Caldera", "Generador", "Maquinaria"]),
                    combustible,
                    round(random.uniform(50, 500), 2),
                    round(random.choice(datos_tecnicos['densidades']), 2)
                ])

            # HC3_Chimeneas.csv (1-2 registros)
            for j in range(random.randint(1, 2)):
                datos['HC3_Chimeneas.csv'].append(datos_base + [
                    f"CH-{j+1:02d}",
                    round(random.uniform(1.0, 1.2), 2),
                    random.randint(120, 250),
                    round(random.uniform(7.5, 9.0), 1),
                    round(random.uniform(100, 300), 1),
                    round(random.uniform(90, 280), 1),
                    random.randint(6000, 8760)
                ])

            # HC4_Descargas.csv (1-3 registros)
            for j in range(random.randint(1, 3)):
                datos['HC4_Descargas.csv'].append(datos_base + [
                    f"D-{j+1:02d}",
                    round(random.uniform(5, 50), 1),
                    random.randint(6000, 8760),
                    random.choice(["Agua residual", "Emision gaseosa", "Liquido proceso"]),
                    random.choice(datos_tecnicos['fluidos']),
                    round(random.uniform(1.0, 5.0), 1)
                ])

            # HC5_Emisiones.csv (2-3 registros)
            for j in range(random.randint(2, 3)):
                datos['HC5_Emisiones.csv'].append(datos_base + [
                    random.choice(datos_tecnicos['emisiones']),
                    random.choice(["Atmosfera", "Suelo", "Agua"]),
                    f"Fuente_{random.randint(1, 5)}",
                    random.choice(["kg/ano", "ton/ano"]),
                    round(random.uniform(50, 50000), 2),
                    random.choice(["Monitoreo", "Factor de emision", "Balance masico"])
                ])

            # HC6_Residuos.csv (2-3 registros)
            for j in range(random.randint(2, 3)):
                datos['HC6_Residuos.csv'].append(datos_base + [
                    datos_tecnicos['residuos'][j % len(datos_tecnicos['residuos'])],
                    round(random.uniform(10, 500), 2),
                    random.choice(datos_tecnicos['unidades']),
                    round(random.choice(datos_tecnicos['densidades']), 2),
                    f"20{random.randint(10000000, 99999999)}",
                    random.choice(["Reciclador", "Gestor autorizado", "Dispositor final"]),
                    f"REG-{empresa['RUC'][-4:]}-{j+1:03d}"
                ])


    # Escribir archivos CSV y comprimir
    with zipfile.ZipFile('RETC_Datos_Originales.zip', 'w') as zipf:
        for archivo, encabezados in ARCHIVOS.items():
            with open(archivo, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(encabezados)
                writer.writerows(datos[archivo])
            zipf.write(archivo)

    print("✓ Archivos generados exitosamente!")
    files.download('RETC_Datos_Originales.zip')


In [8]:
# Ejecución en Colab
print("1. Sube el archivo retc_database_500.txt")
filename = upload_file()

1. Sube el archivo retc_database_500.txt


Saving retc_database_500.txt to retc_database_500.txt
Archivo retc_database_500.txt cargado (33279 bytes)


In [18]:
if filename:
    print("\n2. Procesando datos...")
    empresas = leer_empresas(filename)[:200]
    generar_csvs(empresas)
else:
    print("Error: No se subió ningún archivo")


2. Procesando datos...
Error en línea: Un establecimiento se define como una o varias instalaciones situadas en el mismo emplazamiento o emplazamientos adyacentes cuyo propietario o titular sea la misma persona natural o jurнdica. Ingrese u"
 - list index out of range
✓ Archivos generados exitosamente!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>